In [26]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table
from dash.dependencies import Input, Output
import base64


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient #import used in Pythong CRUD Module
from dash import Dash, callback_context
import animalShelter


###########################
# Data Manipulation / Model
###########################

username = "aacuser"
password = "BLB019640!"
shelter = animalShelter.AnimalShelter() 

# class read method must support return of cursor object and accept projection json input
df = pd.DataFrame.from_records(shelter.search({}))

image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.A([html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),
                    style={'height':'20%','width':'20%'}),], href='https://www.snhu.edu'),
    html.Center(html.B(html.H1('Trice Bonomo SNHU CS-340 Dashboard'))),
    html.Hr(),
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Button(id='submit-button-one',n_clicks=0, children='Water'),
        html.Button(id='submit-button-two', n_clicks=0, children='Wilderness'),
        html.Button(id='submit-button-three', n_clicks=0, children='Disaster'),
        html.Button(id='submit-button-four', n_clicks=0, children='Reset'),
        html.Div(id='container_button')
    ]),
    dash_table.DataTable(
        style_data={      # wrap cell data
            'whiteSpace':'normal',
            'height':'auto',      
        },             
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),        
        editable=False,
        filter_action="native",
        sort_action="native", 
        column_selectable=False,
        row_selectable=False,
        row_deletable=False,
        fixed_rows=True,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=10,
        

    ),
    html.Br(),
    html.Hr(),
        html.A([html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),
                    style={'height':'20%','width':'20%'}),], href='https://www.snhu.edu'),
    html.H1('Trice Bonomo SNHU CS-340 Dashboard', style={'width':'600px'}),
    html.Div(className='row', children=[
    html.Div(
            id='map-id',
            className='col s6',
            ),
    html.Div([
        html.Label(['Analytics']),
        dcc.Dropdown(
        id='my_dropdown',style={'width': '600px'},
       

        options=[
            {'label': 'Breed', 'value': 'breed'},
            {'label': 'Age (in Weeks)', 'value': 'age_upon_outcome_in_weeks'},

        ])
    ]),
    html.Div([
        dcc.Graph(id='the_graph',style={'width': '600px', 'height': '500px'}, className='col s6',)
            ]),
                html.A([html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),
                    style={'height':'20%','width':'20%'}),], href='https://www.snhu.edu'),
    html.H1('Trice Bonomo SNHU CS-340 Dashboard', style={'width':'600px'}),
    ])
])

#############################################
# Interaction Between Components / Controller
#############################################
#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')])

#This callback handles button clicks for submit-button's one and two
@app.callback(Output('datatable-id',"data"),
             [Input('submit-button-one', 'n_clicks'),
              Input('submit-button-two','n_clicks'),
              Input('submit-button-three','n_clicks'),
              Input('submit-button-four','n_clicks')
             ])
def on_click(btn1,btn2,btn3,btn4):
    if (int(btn1)== 0 and int(btn2) == 0 and int(btn3)==0): #default display
        df = pd.DataFrame.from_records(shelter.search({}))
    elif (int(btn1) == 1 and int(btn2) == 0 and int(btn3) == 0 and int(btn4) == 0 ):
        df = pd.DataFrame(list(shelter.search(
            {"breed":{"$in":["Labrador Retriever Mix","Chesapeake Bay Retriever","Newfoundland"]}, 
             "outcome_type":{"$in":["Adoption","Transfer"]},"sex_upon_outcome":"Intact Female","age_upon_outcome_in_weeks":{"$gte":26},
             "age_upon_outcome_in_weeks":{"$lte":156}}))) #display water
    elif (int(btn1) == 1 and int(btn2) == 1 and int(btn3) == 0 and int(btn4) == 0 ):
        df = pd.DataFrame(list(shelter.search(
            {"breed":{"$in":["German Shepherd","Alaskan Malamute","Old English Sheepdog","Siberian Husky","Rottweiler"]},
             "outcome_type":{"$in":["Adoption","Transfer"]},"sex_upon_outcome":"Intact Male","age_upon_outcome_in_weeks":{"$gte":26},
            "age_upon_outcome_in_weeks":{"$lte":156}}))) #display wilderness
    elif (int(btn1) == 1 and int(btn2) == 1 and int(btn3) == 1 and int(btn4) == 0 ):
        df = pd.DataFrame(list(shelter.search(
            {"breed":{"$in":["German Shepherd","Doberman Pinscher","Golden Retriever","Bloodhound","Rottweiler"]},
            "outcome_type":{"$in":["Adoption","Transfer"]},"sex_upon_outcome":"Intact Male","age_upon_outcome_in_weeks":{"$gte":20},
            "age_upon_outcome_in_weeks":{"$lte":300}}))) #display disaster
    elif (int(btn1) == 1 and int(btn2) == 1 and int(btn3) == 1 and int(btn4) == 1 ):
        df = pd.DataFrame(list(shelter.search({}))) #display all
    elif (int(btn1) == 1 and int(btn2) == 1 and int(btn3) == 2 and int(btn4) == 1 ):
        df = pd.DataFrame(list(shelter.search(
            {"breed":{"$in":["German Shepherd","Doberman Pinscher","Golden Retriever","Bloodhound","Rottweiler"]},
             "outcome_type":{"$in":["Adoption","Transfer"]},"sex_upon_outcome":"Intact Male","age_upon_outcome_in_weeks":{"$gte":20},
            "age_upon_outcome_in_weeks":{"$lte":300}}))) #display disaster
    elif (int(btn1) == 1 and int(btn2) == 2 and int(btn3) == 2 and int(btn4) == 1 ):
        df = pd.DataFrame(list(shelter.search(
            {"breed":{"$in":["German Shepherd","Alaskan Malamute","Old English Sheepdog","Siberian Husky","Rottweiler"]},
             "outcome_type":{"$in":["Adoption","Transfer"]},"sex_upon_outcome":"Intact Male","age_upon_outcome_in_weeks":{"$gte":26},
            "age_upon_outcome_in_weeks":{"$lte":156}}))) #display wilderness
    elif (int(btn1) == 2 and int(btn2) == 2 and int(btn3) == 2 and int(btn4) == 1 ):
        df = pd.DataFrame(list(shelter.search(
            {"breed":{"$in":["Labrador Retriever Mix","Chesapeake Bay Retriever","Newfoundland"]}, 
            "outcome_type":{"$in":["Adoption","Transfer"]},"sex_upon_outcome":"Intact Female","age_upon_outcome_in_weeks":{"$gte":26},
             "age_upon_outcome_in_weeks":{"$lte":156}}))) #display water
    elif (int(btn3) > int(btn1)):
        df = pd.DataFrame(list(shelter.search(
            {"breed":{"$in":["Labrador Retriever Mix","Chesapeake Bay Retriever","Newfoundland"]}, 
             "outcome_type":{"$in":["Adoption","Transfer"]},"sex_upon_outcome":"Intact Female","age_upon_outcome_in_weeks":{"$gte":26},
             "age_upon_outcome_in_weeks":{"$lte":156}}))) #display water
    elif (int(btn1) >= int(btn2)):
        df = pd.DataFrame(list(shelter.search(
            {"breed":{"$in":["German Shepherd","Alaskan Malamute","Old English Sheepdog","Siberian Husky","Rottweiler"]},
             "outcome_type":{"$in":["Adoption","Transfer"]},"sex_upon_outcome":"Intact Male","age_upon_outcome_in_weeks":{"$gte":26},
            "age_upon_outcome_in_weeks":{"$lte":156}}))) #display wilderness
    elif (int(btn1) >= int(btn2)):
        df = pd.DataFrame(list(shelter.search(
            {"breed":{"$in":["German Shepherd","Doberman Pinscher","Golden Retriever","Bloodhound","Rottweiler"]},
             "outcome_type":{"$in":["Adoption","Transfer"]},"sex_upon_outcome":"Intact Male","age_upon_outcome_in_weeks":{"$gte":20},
            "age_upon_outcome_in_weeks":{"$lte":300}}))) #display disaster
    elif (int(btn1) >= int(btn2)):
        df = pd.DataFrame(list(shelter.search({}))) #display all
    return df.to_dict('records')

def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

#callback for displaying map
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])


def update_map(viewData):
#Code for geolocation chart
    dff = pd.DataFrame.from_dict(viewData)
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '600px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[30.75,-97.48], children=[
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1,15]) 
                ])
            ])
        ])
    ]

#callback for displaying piechart
@app.callback(
    Output(component_id='the_graph', component_property='figure'),
    [Input(component_id='my_dropdown', component_property='value'),
    Input('datatable-id', "derived_viewport_data")]
)

def update_graph(my_dropdown,viewData):
    dff = pd.DataFrame.from_dict(viewData)

    piechart=px.pie(
            data_frame=dff,
            names=my_dropdown,
            hole=.3,
            )

    return (piechart)

app